In [1]:
import tensorflow as tf
# Setting variables and function
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2
#Evaluating it in a session
sess = tf.Session()
sess.run(x.initializer) # Spell initializer correctly
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [2]:
# Evaluation shorter code
with tf.Session() as sess:
    x.initializer.run() # In this code it's equivalent to tf.get_default_session().run(x.initializer)
    y.initializer.run()
    result = f.eval() # tf.get_default_session().run(f)
print(result)
sess.close()

42


In [3]:
# Even faster
init = tf.global_variables_initializer() # prepare an init node

with tf.Session() as sess:
    init.run() # initializes all variables
    result = f.eval()
print(result)
sess.close()

42


In [4]:
# Using an interactive session, which automatically is set to default
sess = tf.InteractiveSession()
init.run() # with tf.Session() as sess is unnecessary
result = f.eval()
print(result)
sess.close()

42


In [5]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [6]:
# Setting another graph as the default for a bit
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

if x2.graph is graph:
    print(True)
else:
    print(False)
    
if x2.graph is tf.get_default_graph():
    print(True)
else:
    print(False)

True
False


In [7]:
# Demonstration of how tf evalutates.
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())
# tf sees that y depends on x which depends on w, so it evaluates w, then x, then y. These are then reevalutated for Z,
# previous values are not used
sess.close()

10
15


In [8]:
# Only evaluating w and x once
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)
sess.close()

10
15


In [9]:
# Normal Equation
import numpy as np
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) #tf.matmul matrix multiplication

with tf.Session() as sess:
    theta_value = theta.eval()
sess.close()

In [10]:
# Batch Gradient Descent 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

# Manually computing gradients
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32,name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0,1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer =tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# with auto diff gradients = tf.gradients(mse,[theta])[0] 
# without autodiff = 2/m * tf.matmul(tf.transpose(X), error)
training_op = optimizer.minimize(mse)# Without optimizer training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

sess = tf.InteractiveSession()
sess.run(init)

for epoch in range(n_epochs):
    if epoch % 100 == 0:
        print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
best_theta = theta.eval()
sess.close()

Epoch 0 MSE = 9.093209
Epoch 100 MSE = 8.674361
Epoch 200 MSE = 8.278428
Epoch 300 MSE = 7.904021
Epoch 400 MSE = 7.5498457
Epoch 500 MSE = 7.214685
Epoch 600 MSE = 6.8974123
Epoch 700 MSE = 6.596972
Epoch 800 MSE = 6.3123736
Epoch 900 MSE = 6.0426846


In [11]:
# Placeholder nodes
A = tf.placeholder(tf.float32, shape=(None, 3)) # None means any size
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[4,5,6], [7,8,9]]})
print(B_val_1)
print(B_val_2)
sess.close()

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [12]:
# Mini batch
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0,1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer =tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

init = tf.global_variables_initializer()

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size) 
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

sess = tf.InteractiveSession()
sess.run(init)

for epoch in range(n_epochs):
    for batch_index in range(n_batches):
        X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
best_theta = theta.eval()
sess.close()

In [13]:
'''
#Saving models
saver = tf.train.Saver()
for epoch in range(n_epochs):
    if epoch % 100 == 0:
        save_path = saver.save(sess, "file_path")

save_path = saver.save(sess, "file_path")
'''

'\n#Saving models\nsaver = tf.train.Saver()\nfor epoch in range(n_epochs):\n    if epoch % 100 == 0:\n        save_path = saver.save(sess, "file_path")\n\nsave_path = saver.save(sess, "file_path")\n'

In [14]:
'''
# Restoring models
with tf.Session() as sess:
    saver.restore(sess, "file_path")

#Specifically restoring theta under the name weights
saver = tf.train.Saver({"weights": theta})

#Restoring a graph
saver = tf.train.import_meta_graph("file_path")

with tf.Session() as sess:
    saver.restore(sess, "file_path")
'''

'\n# Restoring models\nwith tf.Session() as sess:\n    saver.restore(sess, "file_path")\n\n#Specifically restoring theta under the name weights\nsaver = tf.train.Saver({"weights": theta})\n\n#Restoring a graph\nsaver = tf.train.import_meta_graph("file_path")\n\nwith tf.Session() as sess:\n    saver.restore(sess, "file_path")\n'

In [15]:
# We will visualize things in tensorboard and use a datetime stamp to separate it from other boards
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

# At the end of construction add this
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

'''
# updating execution phase to regularly evaluate mse_summary
for epoch in range(n_epochs):
    for batch_index in range(n_batches):
        X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
        if batch_index % 10 == 0:
            summary_str = mse_summary.eval(feed_dict=[X: X_batch, y: y_batch])
            step = epoch * n_batches + batch_index
            file_writer.add_summary(summary_str, step)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
'''
file_writer.close()

In [20]:
# Using everything
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0,1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer =tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

# Creating name scopes for nodes: Error and MSE
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

init = tf.global_variables_initializer()

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size) 
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

sess = tf.InteractiveSession()
sess.run(init)

for epoch in range(n_epochs):
    for batch_index in range(n_batches):
        X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
        if batch_index % 10 == 0:
            summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
            step = epoch * n_batches + batch_index
            file_writer.add_summary(summary_str, step)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
best_theta = theta.eval()
sess.close()
file_writer.close()

/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


ValueError: Fetch argument <tf.Operation 'GradientDescent' type=NoOp> cannot be interpreted as a Tensor. (Operation name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_theta/ApplyGradientDescent"
 is not an element of this graph.)

In [23]:
# RelU
def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]),1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X,w), b, name="z")
        return tf.maximum(z, threshold, name="relu")

n_features = 3
threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")